In [3]:
# from transformers import BertModel , BartModel
# model = BartModel.from_pretrained("sshleifer/distilbart-cnn-12-6")
# model.save_pretrained("distilbart-cnn-12-6")

import torch
from transformers import BartTokenizer, BartForSequenceClassification

tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = BartForSequenceClassification.from_pretrained("sshleifer/distilbart-cnn-12-6")

Some weights of the model checkpoint at sshleifer/distilbart-cnn-12-6 were not used when initializing BartForSequenceClassification: ['final_logits_bias']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at sshleifer/distilbart-cnn-12-6 and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-strea

In [5]:
model

BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((10

In [12]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [9]:
# summarizer = pipeline("summarization") 
summarizer = model

In [10]:
# URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
# URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"
URL = "https://ndrohith09.hashnode.dev/why-do-farmers-want-to-give-up-farming"

In [13]:
r = requests.get(URL)

In [14]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [15]:
ARTICLE

'ND Rohith\'s Blog ND Rohith\'s Blog Why do farmers want to give up farming? 6 min read Subscribe to my newsletter and never miss my upcoming articles India is a rising economy with tremendous development potential. We have wonderful land that can yield gold if utilized properly. India is now one of the world\'s largest producer of a variety of dry fruits, agricultural-based textile raw materials, roots and tuber crops, pulses, farmed fish, eggs, coconut, sugarcane, and a variety of vegetables. As of 2020-2021, India exported agricultural goods worth almost 20674 million USD. Despite the fact that we are one of the world\'s top agricultural producing countries, our farmers want to give up farming.  According to a poll of 5,000 farm households from 18 states, 76% of farmers would prefer to pursue anything other than farming. 61% of these farmers would prefer to work in cities due to better education, health care, and career opportunities. A substantial majority of farmers reported recur

In [16]:
max_chunk = 500
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [17]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [18]:
len(chunks)

3